In [1]:
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel, PeftConfig
import json

W1102 21:18:16.514000 19076 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("sarvamai/sarvam-1")
model = AutoModelForCausalLM.from_pretrained("sarvamai/sarvam-1")

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 96636be3-95ed-4640-ad61-f9299cc4a3ed)')' thrown while requesting HEAD https://huggingface.co/sarvamai/sarvam-1/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
dataset = load_dataset("json", data_files="8k.jsonl")

In [4]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target'],
        num_rows: 8137
    })
})


In [6]:
tokenizer.pad_token = tokenizer.eos_token

In [7]:
def tokenize_function(data):
    message = f"Correct grammar for: {data['source']}"
    input = message
    output = data['target']
    tokenized_inputs = tokenizer(input, truncation=True, padding='max_length', max_length=128)
    tokenized_outputs = tokenizer(output, truncation=True, padding='max_length', max_length=128)
    tokenized_inputs['labels'] = tokenized_outputs['input_ids']

    return tokenized_inputs

In [8]:
tokenized_data = dataset['train'].map(tokenize_function)

Map:   0%|          | 0/8137 [00:00<?, ? examples/s]

In [9]:
print(tokenized_data)

Dataset({
    features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 8137
})


In [10]:
tokenized_dataset = tokenized_data.train_test_split(test_size=0.2, seed=42)

In [11]:
print(tokenized_dataset)

DatasetDict({
    train: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 6509
    })
    test: Dataset({
        features: ['source', 'target', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1628
    })
})


In [12]:
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [13]:
from trl import SFTTrainer, SFTConfig

training_args = SFTConfig(
    output_dir="./finetuned-hindi",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2, 
    gradient_accumulation_steps=8,
    learning_rate=2e-5,
    num_train_epochs=3,
    fp16=True,
    logging_steps=50,
    save_strategy="epoch",
    eval_strategy="epoch",
    report_to="none",
)

In [14]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    # data_collator=data_collator
)

# Start fine-tuning
trainer.train()

# Save fine-tuned model
trainer.save_model("./finetuned-hindi-model-8k")
tokenizer.save_pretrained("./finetuned-hindi-model-8k")

Truncating train dataset:   0%|          | 0/6509 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1628 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 